In [8]:
import os
import pandas as pd
import numpy as np
from CAPM_Stats import GetICsAndWeights, GetBetasMktAndSpecVols, CalcStats
import json # to dump matrices as json files

os.getcwd()

'/home/mikea/Desktop/INFO5006S/Project'

In [9]:
tables_path = os.path.join(os.getcwd(),"Tables_CSV")
os.listdir(tables_path)

tables_path = os.path.join(os.getcwd(),"Tables_CSV")
os.listdir(tables_path)

# look at only relevant columns
indx_rel_cols = ["Date","Alpha","ICB Sub-Sector", "Gross Market Capitalisation","ALSI New","Index New","TOPI New","RESI New","FINI New","INDI New","PCAP New","SAPY New","ALTI New"]


# read and define tables
indx_tbl = pd.read_csv(os.path.join(tables_path,"tbl_Index_Constituents.csv"),parse_dates = ["Date"] )
indx_tbl = indx_tbl[indx_rel_cols]
beta_tbl = pd.read_csv(os.path.join(tables_path,'tbl_BA_beta_Output.csv'),parse_dates = ["Date"])

In [10]:
indx_tbl.head()

,Date,Alpha,ICB Sub-Sector,Gross Market Capitalisation,ALSI New,Index New,TOPI New,RESI New,FINI New,INDI New,PCAP New,SAPY New,ALTI New
0,2017-09-18,ACE,2353,"101877055,32",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTI
1,2017-09-18,ACG,8771,"889880860,6",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALTI
2,2017-09-18,ACL,1757,"5667537928,5",ALSI,SMLC,NaN,,NaN,NaN,NaN,NaN,NaN
3,2017-09-18,ACS,8633,"3001695817,6",FLED,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-09-18,ACT,4533,"3331807741,28",ALSI,SMLC,NaN,NaN,NaN,,NaN,NaN,NaN


In [11]:
beta_tbl.head()

,Date,Instrument,Index,Start Date,End Date,% Days Traded,Data Points,Alpha,Beta,SE Alpha,SE Beta,p-Value Alpha,p-Value Beta,R2,Total Risk,Unique Risk
0,2017-09-29,ACE,J200,2012-09-28 00:00:00.000,2017-09-27 00:00:00.000,"0,534455128205128",60,"0,0041469017632358","0,317136866849211","0,0176714604719485","0,294306606310179","0,815293981162304","0,28122433536531","0,0081514795412966","0,473269738608869","0,471336867312839"
1,2017-09-29,ACE,J203,2012-09-28 00:00:00.000,2017-09-27 00:00:00.000,"0,534455128205128",60,"0,0037750529378066","0,361932732196224","0,0177553818358639","0,339812397184851","0,832373094945071","0,286832612772359","0,0088813789419179","0,473269738608869","0,471163407451973"
2,2017-09-29,ACE,J250,2012-09-28 00:00:00.000,2017-09-27 00:00:00.000,"0,534455128205128",60,"0,0061690971965583","0,159250221174437","0,0178473738514639","0,358862402170007","0,730849957202203","0,657213206214222","0,0015787823424214","0,473269738608869","0,472895996082816"
3,2017-09-29,ACE,J257,2012-09-28 00:00:00.000,2017-09-27 00:00:00.000,"0,534455128205128",60,"0,0076020068315562","0,010671577180574","0,0178120478791652","0,348898724007764","0,671112248802125","0,975599339734194","8,1788217029E-06","0,473269738608869","0,473267803210507"
4,2017-09-29,ACE,J258,2012-09-28 00:00:00.000,2017-09-27 00:00:00.000,"0,534455128205128",60,"0,0031654059942849","0,245553482475974","0,017870890757908","0,207713641977186","0,860026237458489","0,237137019259714","0,0172183553460152","0,473269738608869","0,469177583812647"


Get all unique combinations of Alpha (index_codes) and Dates

In [12]:
quarters = indx_tbl["Date"].unique()
# convert to datetime because CAPM_Stats works with datetime dates
quarters = pd.to_datetime(quarters)
quarters

DatetimeIndex(['2017-09-18', '2017-12-18', '2018-03-19', '2018-06-18',
               '2018-09-25', '2018-12-24', '2019-03-18', '2019-06-24',
               '2019-09-23', '2019-12-23', '2020-03-23', '2020-06-22'],
              dtype='datetime64[ns]', freq=None)

In [13]:
index_codes = ["ALSI","FLED","LRGC", "MIDC", "SMLC", "TOPI", "RESI", "FINI", "INDI", "PCAP", "SAPY", "ALTI"]

In [14]:
mkt_index_codes = ["J203", "J200", "J250", "J257", "J258"]

Make a dictionary to convert Sub-Sector Code to industry code

In [15]:
Industry = pd.read_csv("Industry.csv")
Industry.head()

,Industry Code,Industry,Super Sector Code,Super Sector,Sector Code,Sector,Sub-Sector Code,Sub-Sector
0,1,Oil & Gas,500,Oil & Gas,530,Oil & Gas Producers,533,Exploration & Production
1,1,Oil & Gas,500,Oil & Gas,530,Oil & Gas Producers,537,Integrated Oil & Gas
2,1,Oil & Gas,500,Oil & Gas,570,"Oil Equipment, Services & Distribution",573,Oil Equipment & Services
3,1,Oil & Gas,500,Oil & Gas,570,"Oil Equipment, Services & Distribution",577,Pipelines
4,1,Oil & Gas,500,Oil & Gas,580,Alternative Energy,583,Renewable Energy Equipment


In [16]:
subsectors = list(Industry["Sub-Sector Code"]) # do not necessarily have to coerce into lists
industry_names = list(Industry["Industry"])


The goal is to make a dictionary that maps share codes to a particular industry to do this first subsector to industry must be mapped. Then share to subsector. Finally the 2 will be combined to map from share to industry

In [17]:
# make dictionary to map each sub sector code to an industry 
subsect2ind = {}
for i in range(len(subsectors)):
    subsect2ind[subsectors[i]] = industry_names[i]

In [18]:
# make dictionary to map share code to industry using previous dictionary
ICB_sub = list(indx_tbl["ICB Sub-Sector"])
share_codes = list(indx_tbl["Alpha"])

share2ind = {}

for i in range(len(share_codes)):
    share2ind[share_codes[i]] = subsect2ind[ICB_sub[i]]




Define dataframes to store the data

In [19]:
shares_metrics = pd.DataFrame(columns = ["date","index_code","mkt_index_code","share_code","industry","weight","beta","spec_vol"])
shares_metrics

,date,index_code,mkt_index_code,share_code,industry,weight,beta,spec_vol


In [20]:
portfolio_metrics = pd.DataFrame(columns = ["date","index_code","mkt_index_code","mkt_vol","port_beta","port_sys_vol","port_spec_vol","port_vol","sys_cov_mat","spec_cov_mat","total_cov_mat","corr_mat","mat_id","shares_excl"])
portfolio_metrics

,date,index_code,mkt_index_code,mkt_vol,port_beta,port_sys_vol,port_spec_vol,port_vol,sys_cov_mat,spec_cov_mat,total_cov_mat,corr_mat,mat_id,shares_excl


Iterate through all index codes and quarters to populate both dataframes. Note all the matrices will be stored as json text and will have an associated matrix ID so that you can identify which 2 shares each element in a matrix refers to

In [21]:
for index in index_codes:
    for quarter in quarters:
        IC, weights = GetICsAndWeights(indx_tbl,quarter,index)
        for mkt_index in mkt_index_codes:
            betas, specVols, mktVol = GetBetasMktAndSpecVols(beta_tbl,quarter,IC,mkt_index)
            
            # use dict to get corresponding industry
            industries = [share2ind[share] for share in IC]
            
            # add shares data to share df
            shares_data = np.column_stack(([quarter]*len(IC),[index]*len(IC),[mkt_index]*len(IC),IC,industries,weights,betas,specVols)) # use to treat arrays as column vectors
            shares_metrics_temp = pd.DataFrame(data = shares_data,columns = shares_metrics.columns)
            shares_metrics = pd.concat([shares_metrics,shares_metrics_temp], ignore_index = True)

            pfBeta, sysCov, pfSysVol, specCov, pfSpecVol, totCov, pfVol, CorrMat, indx_nan = CalcStats(weights, betas, mktVol, specVols)  
            # convert matrices to json format
            json_sysCov = json.dumps(sysCov.tolist()) # remember you have to use tolist() simple type casting does not work for 2D arrays
            json_specCov = json.dumps(specCov.tolist())
            json_totCov = json.dumps(totCov.tolist())
            json_CorrMat = json.dumps(CorrMat.tolist())

            # get list of share codes that were excluded in the portfolio calculations
            shares_excl = IC[indx_nan]

            # make a dictionary to convert matrix indices to share code so that you can see which 2 shares each matrix element refers to
            IC_pf = np.delete(IC,indx_nan)
            keys = list(range(1,len(IC_pf)+1)) #indices from 1 to n
            matID = dict(zip(keys,IC_pf))
            # convert to json
            json_matID = json.dumps(matID)

            # remember shares excl must be coerced into a str
            portfolio_data = np.column_stack((quarter, index, mkt_index, mktVol, pfBeta, pfSysVol, pfSpecVol, pfVol, json_sysCov, json_specCov, json_totCov, json_CorrMat, json_matID, str(shares_excl)))
            portfolio_metrics_temp = pd.DataFrame(data = portfolio_data, columns = portfolio_metrics.columns)
            portfolio_metrics = pd.concat([portfolio_metrics,portfolio_metrics_temp], ignore_index= True)
        
        
        
        
        
        

I got a LinAlgError: Singular matrix for the values ('ALSI', 'J203', Timestamp('2018-12-24 00:00:00')). Used the pseudoinverse to solve the issue or attempt to mitigate the issue in some other way

In [11]:
index,mkt_index, quarter

('ALTI', 'J258', Timestamp('2020-06-22 00:00:00'))

In [22]:
shares_metrics.head()

,date,index_code,mkt_index_code,share_code,industry,weight,beta,spec_vol
0,2017-09-18,ALSI,J203,ACL,Basic Materials,0.000496776,1.09594,0.545651
1,2017-09-18,ALSI,J203,ACT,Health Care,0.000292042,0.689231,0.250324
2,2017-09-18,ALSI,J203,ADH,Consumer Services,0.000801619,0.119885,0.206248
3,2017-09-18,ALSI,J203,AEL,Technology,0.000468199,0.936571,0.360658
4,2017-09-18,ALSI,J203,AFE,Basic Materials,0.00112681,0.650427,0.196659


In [23]:
portfolio_metrics.head()

,date,index_code,mkt_index_code,mkt_vol,port_beta,port_sys_vol,port_spec_vol,port_vol,sys_cov_mat,spec_cov_mat,total_cov_mat,corr_mat,mat_id,shares_excl
0,2017-09-18,ALSI,J203,0.108806,1.01834,0.110802,0.0546594,0.12355,"[[0.014219455782170567, 0.008942536349121009, ...","[[0.2977345345148987, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.3119539902970692, 0.008942536349121009, 0....","[[1.0000000000000002, 0.061270306417929056, 0....","{""1"": ""ACL"", ""2"": ""ACT"", ""3"": ""ADH"", ""4"": ""AEL...",['NRP']
1,2017-09-18,ALSI,J200,0.120835,0.887648,0.107259,0.0545739,0.120345,"[[0.011434636304398168, 0.007983731854482985, ...","[[0.3005193539926715, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.31195399029706966, 0.007983731854482985, 0...","[[1.0000000000000004, 0.054698310282173736, 0....","{""1"": ""ACL"", ""2"": ""ACT"", ""3"": ""ADH"", ""4"": ""AEL...",['NRP']
2,2017-09-18,ALSI,J250,0.104251,0.920511,0.0959639,0.0557165,0.110966,"[[0.00546506163302745, 0.0033748218807229504, ...","[[0.306488928664042, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.31195399029706944, 0.0033748218807229504, ...","[[1.0, 0.0231241182077587, 0.00183217816303477...","{""1"": ""ACL"", ""2"": ""ACT"", ""3"": ""ADH"", ""4"": ""AEL...",['NRP']
3,2017-09-18,ALSI,J257,0.113161,0.820766,0.0928786,0.05478,0.10783,"[[0.003741157381927604, 0.002567764321379915, ...","[[0.30821283291514145, 0.0, 0.0, 0.0, 0.0, 0.0...","[[0.31195399029706905, 0.002567764321379915, 0...","[[1.0, 0.01759375431393992, 0.0042445291961010...","{""1"": ""ACL"", ""2"": ""ACT"", ""3"": ""ADH"", ""4"": ""AEL...",['NRP']
4,2017-09-18,ALSI,J258,0.249578,0.393911,0.0983113,0.0537368,0.112039,"[[0.04871396755897076, 0.019146507035574096, 0...","[[0.2632400227380986, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.3119539902970694, 0.019146507035574096, 0....","[[0.9999999999999994, 0.1312123345244721, 0.00...","{""1"": ""ACL"", ""2"": ""ACT"", ""3"": ""ADH"", ""4"": ""AEL...",['NRP']


Store the 2 tables in csv's 

In [24]:
shares_metrics.to_csv("shares_metrics.csv", index=False) # to get rid of index column
portfolio_metrics.to_csv("portfolio_metrics.csv", index=False)

## Now generate the tables broken down by Industry

In [67]:
industry_portfolio_metrics =  pd.DataFrame(columns = ["date","index","mkt_index","industry","weight","beta","sysVol","specVol","shares_excl"])
industry_portfolio_metrics

,date,index,mkt_index,industry,weight,beta,sysVol,specVol,shares_excl


Construct list of quarters and associated market vol for each index

In [68]:
industry_names_unique = list(set(industry_names))

In [69]:
industry_names_unique

['Technology',
 'Basic Materials',
 'Oil & Gas',
 'Utilities',
 'Health Care',
 'Industrials',
 'Consumer Services',
 'Consumer Goods',
 'Telecommunications',
 'Financials']

In [70]:
# populate industry portfolio metrics
for quarter in quarters:
    for index in index_codes:
        for mkt_index in mkt_index_codes:
            for industry in industry_names_unique:
                # get slice of df corresponding to correct quarter, index and industry
                df_shares = shares_metrics[(shares_metrics["date"] == quarter) & (shares_metrics["mkt_index_code"] == mkt_index) & (shares_metrics["index_code"] == index) & (shares_metrics["industry"] == industry)]
                df_port = portfolio_metrics[(portfolio_metrics["date"] == quarter) & (portfolio_metrics["index_code"] == index) & (portfolio_metrics["mkt_index_code"] == mkt_index)]
                
                # extract values from portfolio table
                mkt_vol = df_port["mkt_vol"].iloc[0,] # only need a single value
                shares_excl = df_port["shares_excl"].iloc[0,]                
                
                # make sure industry is in index for that quarter
                if not df_shares.empty:
                
                    # extract values from shares table
                    # remember to coerce to list first then np array to avoid elements being of type "object"
                    weights = np.array(list(df_shares["weight"]))
                    betas = np.array(list(df_shares["beta"]))
                    specVols = np.array(list(df_shares["spec_vol"]))
                    
                    dummy_pfBeta, dummy_sysCov, pfSysVol, dummy_specCov, pfSpecVol, dummy_totCov, dummy_pfVol, dummy_CorrMat, dummy_indx_nan = CalcStats(weights, betas, mktVol, specVols) 
                    
                    # because we are doing a dot product with the betas and the weights (extracted from shares_metrics table) 
                    # to get the industry beta, we have to ensure no NaNs again
                    
                    # get array containing indexes of nan's from betas and specVols
                    indx_nan = np.where(np.isnan(betas))[0] # returns a tuple of arrays
                    indx_nan = np.append(indx_nan,np.where(np.isnan(specVols))[0]) # in case nans occur at different points
                    indx_nan = list(set(indx_nan)) # get rid of duplicates

                    # delete data points corresponding to nan values
                    weights = np.delete(weights,indx_nan)
                    betas = np.delete(betas,indx_nan)
                    
                    # need to take first index for pfSysVol & pfSpecVol as they are np 1D matrices
                    industry_data = [[quarter, index, mkt_index, industry, sum(weights),sum(weights*betas),pfSysVol[0][0],pfSpecVol[0][0],shares_excl]]
                    industry_portfolio_metrics_temp = pd.DataFrame(data = industry_data, columns = industry_portfolio_metrics.columns)
                    industry_portfolio_metrics = pd.concat([industry_portfolio_metrics,industry_portfolio_metrics_temp], ignore_index = True)
                 
                # industry is not in the quarter
                else:
                    nan = float('NaN')
                    industry_data = [[quarter, index, mkt_index, industry, nan, nan, nan, nan ,shares_excl]]
                    industry_portfolio_metrics_temp = pd.DataFrame(data = industry_data, columns = industry_portfolio_metrics.columns)
                    industry_portfolio_metrics = pd.concat([industry_portfolio_metrics,industry_portfolio_metrics_temp], ignore_index = True)
                    
                
                
            

In [53]:
industry_portfolio_metrics.head()

,date,index,mkt_index,industry,weight,beta,sysVol,specVol,shares_excl
0,2017-09-18,ALSI,J203,Technology,0.114264,0.156384,0.042029,0.024455,['NRP']
1,2017-09-18,ALSI,J203,Basic Materials,0.244449,0.323407,0.086917,0.040129,['NRP']
2,2017-09-18,ALSI,J203,Oil & Gas,NaN,NaN,NaN,NaN,['NRP']
3,2017-09-18,ALSI,J203,Utilities,NaN,NaN,NaN,NaN,['NRP']
4,2017-09-18,ALSI,J203,Health Care,0.027195,0.026766,0.007193,0.003373,['NRP']


Save to csv




In [77]:
industry_portfolio_metrics.to_csv("industry_portfolio_metrics.csv", index = False)b

See maximum number of shares excluded

In [151]:
max_noShares_excl = 0
shares_excl = list(portfolio_metrics["shares_excl"])
for i in range(len(shares_excl)):
    if len(shares_excl[i]) > max_noShares_excl:
        max_noShares_excl = len(shares_excl[i])

max_noShares_excl

7